# Get one symbol nakeds for `NSE`
***

- [x] get equity fno list
- [x] get equity histories
- [x] get index histories
- [x] get lot-size
- [x] get market price of options
- [x] get volatilities for chains
- [x] get option closest to underlying
- [x] get all chains with dte

***

- [ ] get rim SDs based on SDMULT and dte
- [ ] get margins
- [ ] get equity option histories
- [ ] get index option histories

***
 - [ ] pack all into symbol objects



In [202]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

3

# Imports

In [203]:
import json
import math
from datetime import date, datetime, time, timedelta, timezone

import numpy as np
import pytz
import requests
from bs4 import BeautifulSoup
from pandas import json_normalize
from scipy.integrate import quad
from scipy.stats import norm
from tqdm import tqdm

from utils import create_dataclass_from_dict

# Constants

In [204]:
PUTSTDMULT = 1.5
CALLSTDMULT = 1.5

# Helper functions

In [205]:
# HELPER FUNCTIONS
# ****************

def live_cache(app_name):
    """Caches the output for time_out specified. This is done in order to
    prevent hitting live quote requests to NSE too frequently. This wrapper
    will fetch the quote/live result first time and return the same result for
    any calls within 'time_out' seconds.

    Logic:
        key = concat of args
        try:
            cached_value = self._cache[key]
            if now - self._cache['tstamp'] < time_out
                return cached_value['value']
        except AttributeError: # _cache attribute has not been created yet
            self._cache = {}
        finally:
            val = fetch-new-value
            new_value = {'tstamp': now, 'value': val}
            self._cache[key] = new_value
            return val

    """

    def wrapper(self, *args, **kwargs):
        """Wrapper function which calls the function only after the timeout,
        otherwise returns value from the cache.

        """
        # Get key by just concating the list of args and kwargs values and hope
        # that it does not break the code :P
        inputs = [str(a) for a in args] + [str(kwargs[k]) for k in kwargs]
        key = app_name.__name__ + "-".join(inputs)
        now = datetime.now()
        time_out = self.time_out
        try:
            cache_obj = self._cache[key]
            if now - cache_obj["timestamp"] < timedelta(seconds=time_out):
                return cache_obj["value"]
        except:
            self._cache = {}
        value = app_name(self, *args, **kwargs)
        self._cache[key] = {"value": value, "timestamp": now}
        return value

    return wrapper


def split_dates(days: int = 365, chunks: int = 50) -> list:
    """splits dates into buckets, based on chunks"""

    end = datetime.today()
    periods = int(days / chunks)
    start = end - timedelta(days=days)

    if days < chunks:
        date_ranges = [(start, end)]
    else:
        dates = pd.date_range(start, end, periods).date
        date_ranges = list(
            zip(pd.Series(dates), pd.Series(dates).shift(-1) + timedelta(days=-1))
        )[:-1]

    # remove last tuple having period as NaT
    if any(pd.isna(e) for element in date_ranges for e in element):
        date_ranges = date_ranges[:-1]

    return date_ranges


def make_date_range_for_stock_history(
    symbol: str, days: int = 365, chunks: int = 50
) -> list:
    """Uses `split_dates` to make date range for stock history"""

    date_ranges = split_dates(days=days, chunks=chunks)

    series = "EQ"

    ranges = [
        {
            "symbol": symbol,
            "from": start.strftime("%d-%m-%Y"),
            "to": end.strftime("%d-%m-%Y"),
            "series": f'["{series}"]',
        }
        for start, end in date_ranges
    ]

    return ranges


def clean_stock_history(result: list) -> pd.DataFrame:
    """Cleans output of"""

    df = pd.concat(
        [pd.DataFrame(r.get("data")) for r in result], axis=0, ignore_index=True
    )

    # ...clean columns

    mapping = {
        "CH_SYMBOL": "nse_symbol",
        "TIMESTAMP": "date",
        "CH_OPENING_PRICE": "open",
        "CH_TRADE_HIGH_PRICE": "high",
        "CH_TRADE_LOW_PRICE": "low",
        "CH_CLOSING_PRICE": "close",
        "CH_TOT_TRADED_QTY": "qty_traded",
        "CH_TOT_TRADED_VAL": "value_traded",
        "CH_TOTAL_TRADES": "trades",
        "VWAP": "vwap",
        "updatedAt": "extracted_on",
    }

    df = df[[col for col in mapping.keys() if col in df.columns]].rename(
        columns=mapping
    )

    # ...convert column datatypes

    astype_map = {
        **{
            k: "float"
            for k in ["open", "high", "low", "close", "value_traded", "trades", "vwap"]
        },
        **{"qty_traded": "int"},
    }

    df = df.astype(astype_map)

    # ...change date columns to utc

    replace_cols = ["date", "extracted_on"]
    df1 = df[replace_cols].map(lambda x: datetime.fromisoformat(x))
    df = df.assign(date=df1.date, extracted_on=df1.extracted_on)

    return df


def clean_index_history(results: list) -> pd.DataFrame:
    """cleans index history and builds it as a dataframe"""

    df = pd.concat(
        [pd.DataFrame(json.loads(r.get("d"))) for r in results], ignore_index=True
    )

    # clean the df

    # ...drop unnecessary columns

    df = df.drop(df.columns[[0, 1]], axis=1)

    # ...rename
    df.columns = ["nse_symbol", "date", "open", "high", "low", "close"]

    # ...convert nse_symbol to IB's symbol
    df = pd.concat(
        [
            df.nse_symbol.map(
                {"Nifty Bank": "BANKNIFTY", "Nifty 50": "NIFTY50"}
            ).rename("symbol"),
            df,
        ],
        axis=1,
    )

    utc_dates = df.date.apply(lambda x: convert_to_utc_datetime(x, eod=True))

    df = df.assign(date=utc_dates)

    # .....convert ohlc to numeric
    convert_dict = {k: "float" for k in ["open", "high", "low", "close"]}

    df = df.astype(convert_dict)

    # .....sort by date
    df.sort_values(["nse_symbol", "date"], inplace=True, ignore_index=True)

    # .....add extract_date
    now = datetime.now()
    utc_now = now.astimezone(timezone.utc)
    df = df.assign(extracted_on=utc_now)

    return df


def nse2ib(nse_list):
    """Converts nse to ib friendly symbols"""

    subs = {"M&M": "MM", "M&MFIN": "MMFIN", "L&TFH": "LTFH", "NIFTY": "NIFTY50"}

    list_without_percent_sign = list(map(subs.get, nse_list, nse_list))

    # fix length to 9 characters
    ib_equity_fnos = [s[:9] for s in list_without_percent_sign]

    return ib_equity_fnos


def convert_to_utc_datetime(date_string, eod=False):
    """Converts nse date strings to utc datetimes. If eod is chosen 3:30 PM IST is taken."""

    # List of possible date formats
    date_formats = ["%d-%b-%Y", "%d %b %Y", "%Y-%m-%d %H:%M:%S.%f%z"]

    for date_format in date_formats:
        try:
            dt = datetime.strptime(date_string, date_format)

            # If the parsed datetime doesn't have timezone info, assume it's UTC
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=pytz.UTC)
            else:
                # If it has timezone info, convert to UTC
                dt = dt.astimezone(pytz.UTC)

            if eod:
                # Set time to 3:30 PM India time for all formats when eod is True
                india_time = time(hour=15, minute=30)
                india_tz = pytz.timezone("Asia/Kolkata")
                dt = india_tz.localize(datetime.combine(dt.date(), india_time))
                dt = dt.astimezone(pytz.UTC)
            elif dt.time() == time(0, 0):  # If time is midnight (00:00:00)
                # Keep it as midnight UTC
                dt = dt.replace(hour=0, minute=0, second=0, microsecond=0)

            return dt
        except ValueError:
            continue

    # If none of the formats work, raise an error
    raise ValueError(f"Unable to parse date string: {date_string}")


def convert_to_numeric(col: pd.Series):
    """convert to numeric if possible, only for object dtypes"""

    if col.dtype == "object":
        try:
            return pd.to_numeric(col)
        except ValueError:
            return col
    return col


def convert_daily_volatility_to_yearly(daily_volatility, days: float = 252):
    return daily_volatility * math.sqrt(days)


def equity_iv_df(quotes: dict) -> pd.DataFrame:
    """Build a core df with symbol, undPrice, expiry, strike, volatilities, lot and price."""

    flat_data = json_normalize(quotes, sep="-")

    # get lot from quote
    lot = (
        quotes["stocks"][0].get("marketDeptOrderBook").get("tradeInfo").get("marketLot")
    )

    # build the df
    df = pd.DataFrame(flat_data)

    df = pd.DataFrame(
        [
            {
                "symbol": symbol,
                "instrument": quotes.get("stocks")[i]
                .get("metadata")
                .get("instrumentType"),
                "undPrice": stock_price,
                "expiry": quotes.get("stocks")[i].get("metadata").get("expiryDate"),
                "strike": quotes.get("stocks")[i].get("metadata").get("strikePrice"),
                "hv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("annualisedVolatility"),
                "iv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("impliedVolatility"),
                "lot": lot,
                "price": quotes.get("stocks")[i].get("metadata").get("lastPrice"),
            }
            for i in range(len(quotes))
        ]
    )

    # Convert expiry to UTC NSE eod
    df = df.assign(
        expiry=df.expiry.apply(lambda x: convert_to_utc_datetime(x, eod=True))
    )

    # Convert the rest to numeric
    df = df.apply(convert_to_numeric)

    # Change instrument type
    instrument_dict = {
        "Stock": "STK",
        "Options": "OPT",
        "Currency": "FX",
        "Index": "IDX",
        "Futures": "FUT",
    }

    inst = df.instrument.str.split()

    s = inst.apply(lambda x: "".join(instrument_dict[item] for item in x))

    df = df.assign(instrument=s)

    return df


def find_closest_strike(df, above=False):
    """
    Finds the row with the strike closest to the undPrice.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    above (bool): If True, find the closest strike above undPrice. If False, find the closest strike below undPrice.

    Returns:
    pd.DataFrame: A DataFrame with the single row that has the closest strike.
    """
    undPrice = df["undPrice"].iloc[0]  # Get the undPrice from the first row

    if above:
        # Filter for strikes above undPrice
        mask = df["strike"] > undPrice
    else:
        # Filter for strikes below undPrice
        mask = df["strike"] < undPrice

    if not mask.any():
        return pd.DataFrame()  # Return an empty DataFrame if no rows match the criteria

    # Calculate the absolute difference between strike and undPrice
    diff = np.abs(df.loc[mask, "strike"] - undPrice)

    # Find the index of the row with the minimum difference
    closest_index = diff.idxmin()

    # Return the closest row as a DataFrame
    return df.loc[[closest_index]]


def get_dte(s: pd.Series) -> pd.Series:
    """Gets days to expiry. Expects series of UTC timestamps"""

    now_utc = datetime.now(pytz.UTC)
    return (s - now_utc).dt.total_seconds() / (24 * 60 * 60)


def fbfillnas(ser: pd.Series) -> pd.Series:
    """Fills nan in series forwards first and then backwards"""

    s = ser.copy()

    # Find the first non-NaN value
    first_non_nan = s.dropna().iloc[0]

    # Fill first NaN with the first non-NaN value
    s.iloc[0] = first_non_nan

    # Fill remaining NaN values with the next valid value
    s = s.fillna(s.bfill())

    # Fill remaining NaN values with the previous valid value
    s = s.fillna(s.ffill())

    return ser.fillna(s)


def get_a_stdev(iv: float, price: float, dte: float) -> float:
    """Gives 1 Standard Deviation value for annual iv"""

    return iv * price * math.sqrt(dte / 365)


def get_prob(sd):
    """Compute probability of a normal standard deviation

    Arg:
        (sd) as standard deviation
    Returns:
        probability as a float

    """
    prob = quad(lambda x: np.exp(-(x**2) / 2) / np.sqrt(2 * np.pi), -sd, sd)[0]
    return prob


def get_prec(v: float, base: float) -> float:
    """Gives the precision value

    Args:
       (v) as value needing precision in float
       (base) as the base value e.g. 0.05
    Returns:
        the precise value"""

    try:
        output = round(round((v) / base) * base, -int(math.floor(math.log10(base))))
    except Exception:
        output = None

    return output


# Prettify columns to show based on a dictionary map
def pretty_columns(df: pd.DataFrame, col_map: dict) -> list:
    """prettifies columns based on column map dictionary"""
    
    cols = [v for _, v in col_map.items() if v in df.columns]
    return cols

# Classes

In [206]:
# CLASSES
# *******


class Stocks:
    time_out = 5
    base_url = "https://www.nseindia.com/api"
    page_url = "https://www.nseindia.com/get-quotes/equity?symbol=LT"
    _routes = {
        "stock_meta": "/equity-meta-info",
        "stock_quote": "/quote-equity",
        "stock_derivative_quote": "/quote-derivative",
        "market_status": "/marketStatus",
        "chart_data": "/chart-databyindex",
        "market_turnover": "/market-turnover",
        "equity_derivative_turnover": "/equity-stock",
        "all_indices": "/allIndices",
        "live_index": "/equity-stockIndices",
        "index_option_chain": "/option-chain-indices",
        "equity_option_chain": "/option-chain-equities",
        "currency_option_chain": "/option-chain-currency",
        "pre_open_market": "/market-data-pre-open",
        "holiday_list": "/holiday-master?type=trading",
        "stock_history": "/historical/cm/equity",  # added by rkv
    }

    def __init__(self):
        self.s = requests.Session()
        h = {
            "Host": "www.nseindia.com",
            "Referer": "https://www.nseindia.com/get-quotes/equity?symbol=SBIN",
            "X-Requested-With": "XMLHttpRequest",
            "pragma": "no-cache",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
        }
        self.s.headers.update(h)
        self.s.get(self.page_url)

    def get(self, route, payload={}):
        url = self.base_url + self._routes[route]
        r = self.s.get(url, params=payload)
        return r.json()

    @live_cache
    def stock_quote(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_quote", data)

    @live_cache
    def stock_quote_fno(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_derivative_quote", data)

    @live_cache
    def trade_info(self, symbol):
        data = {"symbol": symbol, "section": "trade_info"}
        return self.get("stock_quote", data)

    @live_cache
    def market_status(self):
        return self.get("market_status", {})

    @live_cache
    def chart_data(self, symbol, indices=False):
        data = {"index": symbol + "EQN"}
        if indices:
            data["index"] = symbol
            data["indices"] = "true"
        return self.get("chart_data", data)

    @live_cache
    def tick_data(self, symbol, indices=False):
        return self.chart_data(symbol, indices)

    @live_cache
    def market_turnover(self):
        return self.get("market_turnover")

    @live_cache
    def eq_derivative_turnover(self, type="allcontracts"):
        data = {"index": type}
        return self.get("equity_derivative_turnover", data)

    @live_cache
    def all_indices(self):
        return self.get("all_indices")

    def live_index(self, symbol="NIFTY 50"):
        data = {"index": symbol}
        return self.get("live_index", data)

    @live_cache
    def index_option_chain(self, symbol="NIFTY"):
        data = {"symbol": symbol}
        return self.get("index_option_chain", data)

    @live_cache
    def equities_option_chain(self, symbol):
        data = {"symbol": symbol}
        return self.get("equity_option_chain", data)

    @live_cache
    def currency_option_chain(self, symbol="USDINR"):
        data = {"symbol": symbol}
        return self.get("currency_option_chain", data)

    @live_cache
    def live_fno(self):
        return self.live_index("SECURITIES IN F&O")

    @live_cache
    def pre_open_market(self, key="NIFTY"):
        data = {"key": key}
        return self.get("pre_open_market", data)

    @live_cache
    def holiday_list(self):
        return self.get("holiday_list", {})

    @live_cache
    def stock_history(self, symbol, days: int = 365, chunks: int = 50):

        date_ranges = make_date_range_for_stock_history(symbol, days, chunks)

        result = []
        for dr in date_ranges:
            result.append(self.get("stock_history", dr))

        df = clean_stock_history(result)

        return df


class IDXHistories:

    time_out = 5
    base_url = "https://niftyindices.com"
    url = "https://niftyindices.com/Backpage.aspx/getHistoricaldatatabletoString"

    # prepare `post` header
    post_header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.77 Safari/537.36",
        "Connection": "keep-alive",
        "sec-ch-ua": '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "DNT": "1",
        "X-Requested-With": "XMLHttpRequest",
        "sec-ch-ua-mobile": "?0",
        "Content-Type": "application/json; charset=UTF-8",
        "Origin": "https://niftyindices.com",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Dest": "empty",
        "Referer": "https://niftyindices.com/reports/historical-data",
        "Accept-Language": "en-US,en;q=0.9,hi;q=0.8",
    }

    def __init__(self, days: int = 365) -> None:
        self.s = requests.Session()

        # update session with default headers and get the cookies
        init_header = requests.utils.default_headers()
        init_header.update(
            {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.77 Safari/537.36",
            }
        )
        self.s.headers.update(init_header)
        c = self.s.get(url=self.url)
        self.cookies = c.cookies

    def get(self, payload={}):

        r = self.s.post(
            url=self.url,
            headers=self.post_header,
            cookies=self.cookies,
            data=payload,
            timeout=self.time_out,
        )

        return r.json()

    def make_histories(self, days: int = 365, chunks: int = 50):
        """Makes histories for NIFTY50 and BANKNIFTY, based on number of days provided"""

        date_ranges = split_dates(days=days, chunks=chunks)

        idx_symbols = ["Nifty Bank", "Nifty 50"]

        # organize the payloads
        payloads = [
            {
                "cinfo": str(
                    {
                        "name": idx_symbol,
                        "startDate": s.strftime("%d-%b-%Y"),
                        "endDate": e.strftime("%d-%b-%Y"),
                        "indexName": idx_symbol,
                    }
                )
            }
            for s, e in date_ranges
            for idx_symbol in idx_symbols
        ]
        # get the raw jsons
        results = []

        for payload in tqdm(payloads):
            r = self.get(payload=json.dumps(payload))
            results.append(r)

        df = clean_index_history(results)

        return df


def rbi_tr_to_json(wrapper):
    trs = wrapper.find_all("tr")
    op = {}
    for tr in trs:
        tds = tr.find_all("td")
        if len(tds) >= 2:
            key = tds[0].text.strip()
            val = tds[1].text.replace(":", "").replace("*", "").replace("#", "").strip()

            op[key] = val
    return op


class RBI:
    base_url = "https://www.rbi.org.in/"

    def __init__(self):
        self.s = requests.Session()

    def current_rates(self):
        r = self.s.get(self.base_url)

        bs = BeautifulSoup(r.text, "html.parser")
        wrapper = bs.find("div", {"id": "wrapper"})

        return rbi_tr_to_json(wrapper)

    def repo_rate(self):

        rate = self.current_rates().get("Policy Repo Rate")[:-1]

        return float(rate)

# Stock FnOs

## Initialize `Stocks` class

In [207]:
nse = Stocks()

## Get underlying price

In [208]:
symbol = "KOTAKBANK"

q = nse.stock_quote(symbol)
stock_price = q.get("priceInfo").get("lastPrice")

## Get equity fno list

In [209]:
equities = nse.live_fno()

# Equities set
fno_equities = {kv.get("symbol") for kv in equities.get("data")}

## Get a stock quote

In [210]:
quotes = nse.stock_quote_fno(symbol)

### Get lot from a stock quote

In [211]:
lot = quotes["stocks"][0].get("marketDeptOrderBook").get("tradeInfo").get("marketLot")

### Get hv and iv with lot from fno quotes

In [212]:
df_vola = equity_iv_df(quotes)
df_vola = df_vola[df_vola.instrument == "STKOPT"]

In [213]:
df_vola.head()

,symbol,instrument,undPrice,expiry,strike,hv,iv,lot,price
1,KOTAKBANK,STKOPT,1806.9,2024-07-25 10:00:00+00:00,1800,28.59,30.82,400,29.40
2,KOTAKBANK,STKOPT,1806.9,2024-07-25 10:00:00+00:00,1800,28.59,27.57,400,37.10
3,KOTAKBANK,STKOPT,1806.9,2024-07-25 10:00:00+00:00,1820,28.59,28.14,400,27.80
4,KOTAKBANK,STKOPT,1806.9,2024-07-25 10:00:00+00:00,1830,28.59,28.39,400,23.85
5,KOTAKBANK,STKOPT,1806.9,2024-07-25 10:00:00+00:00,1900,28.59,31.45,400,8.10


### Find strike closest to underlying

In [214]:
closest_to_und = find_closest_strike(df_vola, above=False)
closest_to_und

,symbol,instrument,undPrice,expiry,strike,hv,iv,lot,price
1,KOTAKBANK,STKOPT,1806.9,2024-07-25 10:00:00+00:00,1800,28.59,30.82,400,29.4


### Get underlying volatlities

In [215]:
volatilities = next(iter(quotes.get("underlyingInfo").get("volatility")))

und_vols = {
    k: convert_daily_volatility_to_yearly(float(v)) for k, v in volatilities.items()
}

und_vols

{'maxVolatility': 24.44674211423682,
 'minVolatility': 19.52564467565668,
 'avgVolatility': 21.27184054095931}

# Stock History

## Get Stock History

In [216]:
df = nse.stock_history(symbol, days=365, chunks=50)

df.head()

,nse_symbol,date,open,high,low,close,qty_traded,value_traded,trades,vwap,extracted_on
0,KOTAKBANK,2023-09-14 18:30:00+00:00,1816.2,1823.45,1811.00,1813.90,4121540,7.485812e+09,127870.0,1816.27,2023-09-15 12:01:03.542000+00:00
1,KOTAKBANK,2023-09-13 18:30:00+00:00,1828.0,1831.75,1814.00,1821.55,2559423,4.661883e+09,150951.0,1821.46,2023-09-14 12:01:03.340000+00:00
2,KOTAKBANK,2023-09-12 18:30:00+00:00,1800.6,1836.00,1800.60,1824.60,5931870,1.082715e+10,146715.0,1825.25,2023-09-13 12:01:03.097000+00:00
3,KOTAKBANK,2023-09-11 18:30:00+00:00,1819.0,1819.00,1796.40,1811.20,3408966,6.162445e+09,162316.0,1807.72,2023-09-12 12:01:03.087000+00:00
4,KOTAKBANK,2023-09-10 18:30:00+00:00,1800.0,1811.50,1795.25,1807.90,2316902,4.184527e+09,133403.0,1806.09,2023-09-11 12:01:03.166000+00:00


# Index History
 - Index histories are treated in a separate class as:
     - its URL (https://niftyindices.com) is different than equity
     - which needs a POST after of a GET request
     - and also has its date string with a dash (20-Jul-2024)

## Initialize Index Histories

In [217]:
idx = IDXHistories()

df_idx_hist = idx.make_histories(20)  # give number of days needed

df_idx_hist.groupby("symbol").head(3)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.14it/s]


,symbol,nse_symbol,date,open,high,low,close,extracted_on
0,NIFTY50,Nifty 50,2024-06-27 10:00:00+00:00,23881.55,24087.45,23805.40,24044.50,2024-07-17 09:42:21.794676+00:00
1,NIFTY50,Nifty 50,2024-06-28 10:00:00+00:00,24085.90,24174.00,23985.80,24010.60,2024-07-17 09:42:21.794676+00:00
2,NIFTY50,Nifty 50,2024-07-01 10:00:00+00:00,23992.95,24164.00,23992.70,24141.95,2024-07-17 09:42:21.794676+00:00
14,BANKNIFTY,Nifty Bank,2024-06-27 10:00:00+00:00,52980.30,53180.75,52639.00,52811.30,2024-07-17 09:42:21.794676+00:00
15,BANKNIFTY,Nifty Bank,2024-06-28 10:00:00+00:00,52874.95,53030.30,52242.30,52342.25,2024-07-17 09:42:21.794676+00:00
16,BANKNIFTY,Nifty Bank,2024-07-01 10:00:00+00:00,52351.15,52656.15,52166.05,52574.75,2024-07-17 09:42:21.794676+00:00


# Get All chains

In [233]:
stock_chain = nse.equities_option_chain(symbol)  # Equity option chains
data = stock_chain.get("records").get("data")

# idx_chain = nse.index_option_chain("NIFTY")  # Index Option chains
# data = idx_chain.get('records').get('data')

pe = [data[i].get("PE") for i in range(len(data))]
df_pe = json_normalize(pe, sep="-").dropna(subset=["identifier"])
# df_pe = df_pe.sort_values('strikePrice', ascending=False).reset_index(drop=True)
df_pe["right"] = "P"

ce = [data[i].get("CE") for i in range(len(data))]
df_ce = json_normalize(ce, sep="-").dropna(subset=["identifier"])
df_ce["right"] = "C"
# df_ce = df_ce.sort_values('strikePrice', ascending=False).reset_index(drop=True)

df = pd.concat([df_ce, df_pe], ignore_index=True)

chain_col_map = {
    "underlying": "symbol",
    "underlyingValue": "undPrice",
    "expiryDate": "expiry",
    "strikePrice": "strike",
    "right": "right",
    "openInterest": "oi",
    "changeinOpenInterest": "oi_chg",
    "pchangeinOpenInterest": "oi_pchg",
    "totalTradedVolume": "volume",
    "change": "change",
    "pChange": "ltp_pct_chg",
    "totalBuyQuantity": "buy_qty",
    "totalSellQuantity": "sell_qty",
    "bidQty": "bid_qty",
    "bidprice": "bid",
    "askQty": "ask_qty",
    "askPrice": "ask",
    "impliedVolatility": "iv",
    "dte": "dte",
    "stdev": "stdev",
    "sigma": "sigma",
    "safe_strike": "safe_strike",
    "lastPrice": "ltp",
    "xPrice": "xPrice",
    "lot": "lot",
}

df.rename(columns=chain_col_map, inplace=True, errors="ignore")

# Convert expiry to UTC NSE eod
df = df.assign(expiry=df.expiry.apply(lambda x: convert_to_utc_datetime(x, eod=True)))

# Convert the rest to numeric
df = df.apply(convert_to_numeric)

# Replace zero values of iv and ltp to np.nan
df["iv"] = df["iv"].replace(0, np.nan)
df["ltp"] = df["ltp"].replace(0, np.nan)

# Gets dte
df["dte"] = get_dte(df.expiry)

# Sort the columns
df = df.sort_values(
    ["dte", "right", "strike"], ascending=[True, True, False]
).reset_index(drop=True)

# Fill missing ivs
df.iv = fbfillnas(df.iv)

# Put the lots
df = df.assign(lot = lot)

# Put the stdev

# Prettify the columns
cols = pretty_columns(df, chain_col_map)
df = df[cols]

# Experiments for stdev fencing

## Get PUT and CALL option prices using black scholes on option's iv

In [234]:
# Get the risk free rate
rbi = RBI()
risk_free_rate = rbi.repo_rate()

# Get sigma from iv, based on call or put stdmult
df = df.assign(sigma = np.where(df.right=='C', df.iv*CALLSTDMULT, df.iv*PUTSTDMULT))

In [235]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Black-Scholes Option Pricing Model
def black_scholes(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'C':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'P':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'C' for Call and 'P' for Put.")
    
    return price


In [236]:
# Compute the expected price
xPrice = df.apply(lambda row: black_scholes(
    S=row['undPrice'],
    K=row['strike'],
    T=row['dte'] / 365,  # Convert days to years
    r=risk_free_rate/100,
    sigma=row['sigma'] / 100,  # Convert percentage to decimal
    option_type=row['right']
), axis=1)

In [237]:
# Adjust the expected price precision
df = df.assign(xPrice = xPrice.apply(lambda x: get_prec(x, base=0.05)))

# Get the stdev of sigma. This is used to check the risk.
sigmaSTD = [get_a_stdev(sigma/100, S, T) for sigma, S, T in zip(df.iv, df.undPrice, df.dte)]
safe_strike = np.where(df.right == "C", df.undPrice + sigmaSTD, df.undPrice - sigmaSTD)
df = df.assign(safe_strike = safe_strike )

In [238]:
# Sort based on the juciest xPrice:ltp
df = df.iloc[(df.xPrice/df.ltp).sort_values().index]
df = df[df.xPrice >0] # remove zero xPrices


In [239]:
df

,symbol,undPrice,expiry,strike,right,oi,oi_chg,oi_pchg,volume,change,ltp_pct_chg,buy_qty,sell_qty,bid_qty,bid,ask_qty,ask,iv,dte,ltp,lot,sigma,xPrice,safe_strike
30,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1710.0,C,9.0,0.0,0.0,0.0,0.0,0.0,20800.0,28400.0,400.0,100.90,400.0,107.15,25.58,8.007606,128.25,400,38.370,107.75,1875.360424
38,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1600.0,C,29.0,0.0,0.0,0.0,0.0,0.0,32800.0,50800.0,400.0,204.75,2800.0,214.95,41.07,8.007606,240.25,400,61.605,215.55,1916.816716
42,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1480.0,C,2.0,0.0,0.0,0.0,0.0,0.0,16000.0,16000.0,16000.0,277.15,16000.0,356.90,41.07,8.007606,366.00,400,61.605,329.75,1916.816716
28,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1730.0,C,68.0,0.0,0.0,0.0,0.0,0.0,25600.0,28400.0,1200.0,82.45,400.0,89.65,42.35,8.007606,122.75,400,63.525,113.40,1920.242414
33,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1660.0,C,10.0,0.0,0.0,0.0,0.0,0.0,24800.0,25600.0,400.0,144.40,1200.0,153.45,41.07,8.007606,175.50,400,61.605,163.85,1916.816716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,KOTAKBANK,1806.9,2024-08-29 10:00:00+00:00,1520.0,P,0.0,0.0,0.0,0.0,0.0,0.0,40400.0,400.0,1200.0,1.20,400.0,2.90,21.16,43.007606,NaN,400,31.740,3.65,1675.657050
145,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1880.0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,0.0,0.00,400.0,111.00,21.16,71.007606,NaN,400,31.740,79.35,1975.538133
146,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1840.0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,21.16,71.007606,NaN,400,31.740,96.20,1975.538133
147,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1800.0,C,0.0,0.0,0.0,0.0,0.0,0.0,400.0,0.0,400.0,80.05,0.0,0.00,21.16,71.007606,NaN,400,31.740,115.45,1975.538133


In [252]:
mask = (df.undPrice > df.safe_strike) & (df.right == 'C') * (df.dte < 9)
df[mask].head()

,symbol,undPrice,expiry,strike,right,oi,oi_chg,oi_pchg,volume,change,ltp_pct_chg,buy_qty,sell_qty,bid_qty,bid,ask_qty,ask,iv,dte,ltp,lot,sigma,xPrice,safe_strike


In [256]:
df.loc[(df.safe_strike/df.undPrice).sort_values().index]

,symbol,undPrice,expiry,strike,right,oi,oi_chg,oi_pchg,volume,change,ltp_pct_chg,buy_qty,sell_qty,bid_qty,bid,ask_qty,ask,iv,dte,ltp,lot,sigma,xPrice,safe_strike
150,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1600.0,P,29.0,0.0,0.000000,1.0,0.25,2.857143,8800.0,5600.0,800.0,6.80,400.0,9.25,25.00,71.007606,9.0,400,37.500,32.85,1607.658349
149,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1720.0,P,0.0,0.0,0.000000,0.0,0.00,0.000000,800.0,0.0,800.0,23.40,0.0,0.00,25.00,71.007606,NaN,400,37.500,69.25,1607.658349
90,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1400.0,P,449.0,-63.0,-12.304688,75.0,-0.10,-20.000000,81200.0,118800.0,400.0,0.35,400.0,0.40,67.16,8.007606,0.4,400,100.740,4.15,1627.157933
148,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1800.0,P,4.0,1.0,33.333333,1.0,3.00,6.666667,2400.0,2000.0,400.0,52.00,400.0,99.95,21.16,71.007606,48.0,400,31.740,85.95,1638.261867
140,KOTAKBANK,1806.9,2024-08-29 10:00:00+00:00,1620.0,P,0.0,0.0,0.000000,0.0,0.00,0.000000,89600.0,4000.0,4000.0,3.25,1200.0,16.10,27.02,43.007606,NaN,400,40.530,26.50,1639.310940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,KOTAKBANK,1806.9,2024-08-29 10:00:00+00:00,1600.0,C,0.0,0.0,0.000000,0.0,0.00,0.000000,27600.0,28800.0,3600.0,216.05,1200.0,234.35,25.77,43.007606,NaN,400,38.655,238.75,1966.736051
97,KOTAKBANK,1806.9,2024-08-29 10:00:00+00:00,1940.0,C,1.0,0.0,0.000000,1.0,1.80,6.923077,22400.0,20000.0,400.0,10.00,16000.0,37.60,26.90,43.007606,27.8,400,40.350,54.45,1973.744772
145,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1880.0,C,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,400.0,0.0,0.00,400.0,111.00,21.16,71.007606,NaN,400,31.740,79.35,1975.538133
146,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1840.0,C,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,21.16,71.007606,NaN,400,31.740,96.20,1975.538133


In [251]:
df.assign(rom=df.ltp/df.undPrice*14/100*df.lot*365/df.dte)

,symbol,undPrice,expiry,strike,right,oi,oi_chg,oi_pchg,volume,change,ltp_pct_chg,buy_qty,sell_qty,bid_qty,bid,ask_qty,ask,iv,dte,ltp,lot,sigma,xPrice,safe_strike,rom
30,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1710.0,C,9.0,0.0,0.0,0.0,0.0,0.0,20800.0,28400.0,400.0,100.90,400.0,107.15,25.58,8.007606,128.25,400,38.370,107.75,1875.360424,181.176319
38,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1600.0,C,29.0,0.0,0.0,0.0,0.0,0.0,32800.0,50800.0,400.0,204.75,2800.0,214.95,41.07,8.007606,240.25,400,61.605,215.55,1916.816716,339.396575
42,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1480.0,C,2.0,0.0,0.0,0.0,0.0,0.0,16000.0,16000.0,16000.0,277.15,16000.0,356.90,41.07,8.007606,366.00,400,61.605,329.75,1916.816716,517.041192
28,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1730.0,C,68.0,0.0,0.0,0.0,0.0,0.0,25600.0,28400.0,1200.0,82.45,400.0,89.65,42.35,8.007606,122.75,400,63.525,113.40,1920.242414,173.406575
33,KOTAKBANK,1806.9,2024-07-25 10:00:00+00:00,1660.0,C,10.0,0.0,0.0,0.0,0.0,0.0,24800.0,25600.0,400.0,144.40,1200.0,153.45,41.07,8.007606,175.50,400,61.605,163.85,1916.816716,247.925490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,KOTAKBANK,1806.9,2024-08-29 10:00:00+00:00,1520.0,P,0.0,0.0,0.0,0.0,0.0,0.0,40400.0,400.0,1200.0,1.20,400.0,2.90,21.16,43.007606,NaN,400,31.740,3.65,1675.657050,NaN
145,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1880.0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,0.0,0.00,400.0,111.00,21.16,71.007606,NaN,400,31.740,79.35,1975.538133,NaN
146,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1840.0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,21.16,71.007606,NaN,400,31.740,96.20,1975.538133,NaN
147,KOTAKBANK,1806.9,2024-09-26 10:00:00+00:00,1800.0,C,0.0,0.0,0.0,0.0,0.0,0.0,400.0,0.0,400.0,80.05,0.0,0.00,21.16,71.007606,NaN,400,31.740,115.45,1975.538133,NaN


In [248]:
100842/1806.9/400

0.13952349327577618

In [183]:
undPrice = 2820
strike = 3080
right = 'C'

-260

In [182]:
margin = 219780
dte = 43.048046
price=112.45
price*lot/margin*365/dte

1.3014634224154398

In [201]:
6855/228380*365/8

1.3694691960767142

In [123]:
882.55-68

814.55

In [50]:
# Split df calls / puts with strikes above / below undPrice
c_mask = (df.right == "C") & (df.strike > df.undPrice)
dfc = df[c_mask]

p_mask = (df.right == "P") & (df.strike <= df.undPrice)
dfp = df[p_mask]


In [ ]:

# Risk-free rate
r = 0.065  # 6.5%

# Calculate option prices using Black-Scholes model
df['xPrice'] = df.apply(lambda row: black_scholes(
    S=row['undPrice'],
    K=row['strike'],
    T=row['dte'] / 365,  # Convert days to years
    r=r,
    sigma=row['iv'] / 100,  # Convert percentage to decimal
    option_type=row['right']
), axis=1)

# Round xPrice to 2 decimal places for readability
df['xPrice'] = df['xPrice'].round(2)

df

In [ ]:
cols = ['symbol', 'undPrice', 'strike', 'dte', 'right', 'iv']
pd.concat((dfc[cols].sample(3), dfp[cols].sample(3)), ignore_index=True).to_dict('records')

In [ ]:
dfc = dfc.assign(xPrice=get_call_prices(dfc))
dfp = dfp.assign(xPrice=get_put_prices(dfp))

In [ ]:
risk_free_rate

In [ ]:
# dfp[~dfp.xPrice.isnull()]
dfp[chain_cols].head(1)

In [ ]:
call_price = black_scholes_call(
    spot_price, strike_price, time_to_expiry, risk_free_rate, implied_volatility
)
put_price = (
    call_price
    - spot_price
    + strike_price * np.exp(-risk_free_rate * time_to_expiry)
)

In [ ]:
black_scholes_put(882, 880, 8.141136/365, risk_free_rate*PUTSTDMULT, 29.66)

In [ ]:
# Get expected prices from black-scholes
df = pd.concat((dfc, dfp), ignore_index=True)

# Choose columns to show
chain_cols = [v for _, v in chain_col_map.items() if v in df.columns]
df = df[chain_cols]
df

In [ ]:
# Finding the closest above CALLSTDMULT rows
# ... compute the maximum stdev for the dte
std_max = max((dfc.strike - dfc.undPrice) / dfc.stdev)
std_max = max(std_max, CALLSTDMULT)  # ensure it meets the STDMULT threshold

In [ ]:
from scipy.stats import norm


def black_scholes_call(
    spot_price, strike_price, time_to_expiry, risk_free_rate, implied_volatility
):
    """
    This function calculates the theoretical value of a European call option using the Black-Scholes model.

    Args:
        spot_price (float): The current price of the underlying asset.
        strike_price (float): The strike price of the option.
        time_to_expiry (float): The time to expiration of the option in years.
        risk_free_rate (float): The risk-free interest rate in percentage.
        implied_volatility (float): The implied volatility of the underlying asset in percentage.

    Returns:
        float: The theoretical value of the call option.
    """

    # Convert implied volatility and risk-free rate from percentages to decimals
    implied_volatility /= 100
    risk_free_rate /= 100

    # Calculate d1 and d2
    d1 = (
        math.log(spot_price / strike_price)
        + (risk_free_rate + 0.5 * implied_volatility**2) * time_to_expiry
    ) / (implied_volatility * math.sqrt(time_to_expiry))
    d2 = d1 - implied_volatility * math.sqrt(time_to_expiry)

    # Calculate the call option value
    call_value = spot_price * norm.cdf(d1) - strike_price * math.exp(
        -risk_free_rate * time_to_expiry
    ) * norm.cdf(d2)

    return call_value

In [ ]:
dfc

In [ ]:
def get_closest_values(myArr: list, myNumber: float, how_many: int = 0):
    """Get closest values in a list

    how_many: 0 gives the closest value\n
              1 | 2 | ... use for CALL fences\n
             -1 | -2 | ... use for PUT fences"""

    i = 0

    result = dict()

    while i <= abs(how_many):

        if how_many >= 0:
            # going right
            val = myArr[myArr > myNumber].min()
            idx = np.where(myArr[myArr > myNumber] == val)[0][0]
            # print({'val': val, 'idx': idx})

        elif how_many <= 0:
            # going left
            val = myArr[myArr < myNumber].max()
            idx = np.where(myArr[myArr < myNumber] == val)[0][0]
            # print({'val': val, 'idx': idx})

        # else:
        #     val = min(myArr, key=lambda x: abs(x-myNumber))

        # idx = np.where(myArr == val)[0][0]

        result[val] = idx
        myNumber = val
        i += 1

    # output = list(result)[0:abs(1 if how_many == 0 else abs(how_many))]
    output = result

    return output

In [ ]:
def find_indices(arr, my_num, how_many):
    # Convert arr to numpy array if it's not already
    arr = np.array(arr)

    if how_many > 0:
        # Find indices of numbers greater than or equal to my_num
        indices = np.where(arr >= my_num)[0]
        # Sort indices based on the difference from my_num (ascending order)
        indices = indices[np.argsort(np.abs(arr[indices] - my_num))]
        return indices[: min(how_many, len(indices))]

    elif how_many < 0:
        # Find indices of numbers less than or equal to my_num
        indices = np.where(arr <= my_num)[0]
        # Sort indices based on the difference from my_num (ascending order)
        indices = indices[np.argsort(np.abs(arr[indices] - my_num))]
        return indices[: min(abs(how_many), len(indices))]

    else:  # how_many == 0
        # Find index of the closest value to my_num
        return np.argmin(np.abs(arr - my_num))

In [ ]:
mask = (df.right == "C") & (df.strike > df.undPrice)

dfc = df[mask].sort_values(["dte", "strike"], ascending=[True, False])
df1 = dfc[dfc.dte == dfc.dte.min()][chain_cols]
(df1.strike / (df1.undPrice + df1.stdev)).apply(get_prob)

In [ ]:
get_prob(1.5)

In [ ]:
closest_devs = dfc.groupby("dte").stdev.apply(
    lambda x: find_indices(x, CALLSTDMULT, 2)[0]
)
closest_devs

In [ ]:
CALLSTDMULT

In [ ]:
myArr = dfc.stdev
myNumber = CALLSTDMULT
how_many = 1

val = myArr[myArr > myNumber].min()
idx = np.where(myArr == val)[0][0]

In [ ]:
dfcs = dfc[dfc.stdev.isin(closest_devs.to_list())]

In [ ]:
strike = list(range(100, 150, 5))[1:] * 2

In [ ]:
dte = [5, 10] * int(len(strike) / 2)

In [ ]:
size = len(strike)
random_numbers = np.random.rand(size)

nan_probability = 0.3
nan_mask = np.random.random(size) < nan_probability
iv = np.where(nan_mask, np.nan, random_numbers)

p_choice = 0.5
c_choice = 1 - p_choice

right = np.random.choice(["P", "C"], size, p=[p_choice, c_choice])

marker = 0.3

In [ ]:
df = pd.DataFrame({"strike": strike, "dte": dte, "iv": iv, "right": right})

In [ ]:
def choose_closest_iv(df, marker):
    """Chooses the closest iv value to marker for each group in df.

    Args:
        df (pd.DataFrame): Dataframe containing columns 'strike', 'dte', 'iv', and 'right'.
        marker (float): The value to which the closest iv should be chosen.

    Returns:
        pd.DataFrame: A new DataFrame with the same columns as the input df,
                      but with the 'iv' column replaced by the closest values.
    """

    def g(group):
        """Function applied to each group in df.

        Args:
            group (pd.DataFrame): A subgroup of df based on 'dte' and 'right'.

        Returns:
            pd.DataFrame: The group with the 'iv' column replaced by the closest values.
        """
        if group["right"].iloc[0] == "P":
            # For Put options, choose the closest value below marker
            return (
                group[group["iv"] <= marker]
                .sort_values(by="iv", ascending=False)
                .head(1)
            )
        else:
            # For Call options, choose the closest value above marker
            return (
                group[group["iv"] >= marker]
                .sort_values(by="iv", ascending=True)
                .head(1)
            )

    return df.groupby(["dte", "right"]).apply(g)

In [ ]:
choose_closest_iv(df.copy(), marker)

# .... STOPPED HERE for `RIM` options

In [ ]:
idx_chain = nse.index_option_chain("NIFTY")  # Index Option chains
curr_option_chain = nse.currency_option_chain("USDINR")  # Currency option chains

# Option Chains

# Miscellaneous

## Convert nse to ib friendly symbols

In [ ]:
raw_fnos = nse.live_fno()
fno_list = {data.get("symbol") for data in raw_fnos.get("data")}